In [44]:
import pandas as pd
import warnings
import geocoder
import time
import folium
warnings.filterwarnings('ignore')
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.exc import GeocoderTimedOut, GeocoderQuotaExceeded

In [45]:
FP2 = pd.read_csv('Desktop/test/FP2.csv')

In [52]:
sub_prog = FP2[(FP2['programme'] == 'FP2-ECLAIR')] #Geocode limits the number of calls. Only a sample of the database can be retained

In [53]:
Address_info = sub_prog[['street','postCode','city','country']].copy()
Address_info = Address_info.apply(lambda x: x.str.strip(), axis=1)
Address_info['Full_Address'] = Address_info[Address_info.columns[2:5]].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)
locator = Nominatim(user_agent="myGeocoder")  # holds the Geocoding service, Nominatim

In [54]:
geocode = RateLimiter(locator.geocode, min_delay_seconds=1) # convenient function to delay between geocoding calls
def geocode_me(location):
    time.sleep(1.1)
    try:
        return locator.geocode(location)
    except (GeocoderTimedOut, GeocoderQuotaExceeded) as e:
        if GeocoderQuotaExceeded:
            print(e)
        else:
            print(f'Location not found: {e}')
            return None

In [55]:
Address_info['location'] = Address_info['Full_Address'].apply(lambda x: geocode_me(x))
Address_info['point'] = Address_info['location'].apply(lambda loc: tuple(loc.point) if loc else None)
Address_info[['latitude', 'longitude', 'altitude']] =  pd.DataFrame(Address_info['point'].tolist(), index=Address_info.index)
# If you get error message, then your database is too big to be processed by Geocode in one go

In [56]:
Address_info = Address_info.loc[:,~Address_info.columns.isin(['point','location','Full_Address'])]
sub_prog_2 = pd.concat([sub_prog, Address_info], axis=1)

In [57]:
# If you just need the geodatas without the visualization process:
# sub_prog_2.to_csv("Desktop/test/sub_prog_2.csv",mode = 'w', index=False)

In [73]:
# Geographic Visualization in Python
visual = sub_prog_2.dropna(subset=['longitude', 'altitude', 'latitude']) 

In [76]:
map1 = folium.Map(
    location=[50.8333,4.3550], #Brussels' geodatas
    tiles='cartodbpositron',
    zoom_start=4,
)
visual.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1